In [ ]:
# -*- coding: utf-8 -*-
"""
Análisis de Datos: Patrones de Sueño y Estilo de Vida
Autor: Josué Miranda G.
Descripción: EDA, limpieza, outliers (IQR), creación de target binario,
matriz de correlación, división estratificada 80/20 y guardado de train/test.
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# ---------------------------
# Configuración
# ---------------------------
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

os.makedirs('imagenes', exist_ok=True)
os.makedirs('datos', exist_ok=True)

# ---------------------------
# CARGA: ajustar nombre si lo descarga de Kaggle
# ---------------------------
# En Kaggle el archivo suele llamarse algo como:
# "Sleep_health_and_lifestyle_dataset.csv" o "sleep_data.csv".
# Descarga manual o usando kaggle-cli y pon el path correcto aquí.
csv_paths_to_try = [
    'datos/sleep_data.csv',
    'datos/Sleep_health_and_lifestyle_dataset.csv',
    'sleep/Sleep_health_and_lifestyle_dataset.csv',
    'Sleep_health_and_lifestyle_dataset.csv'
]

for p in csv_paths_to_try:
    if os.path.exists(p):
        df = pd.read_csv(p)
        print(f"📥 Cargado: {p}")
        break
else:
    raise FileNotFoundError(
        "No se encontró el CSV. Descargue el dataset de Kaggle y coloque el CSV en 'datos/'. "
        "Nombre esperado: Sleep_health_and_lifestyle_dataset.csv o sleep_data.csv"
    )

# ---------------------------
# Exploración inicial
# ---------------------------
print("\nDimensiones:", df.shape)
display(df.head(5))
print("\nInfo de columnas:")
print(df.info())
print("\nResumen estadístico (numéricas):")
display(df.describe(include=[np.number]).T)
print("\nValores nulos por columna:")
display(df.isnull().sum())

# ---------------------------
# Tratamiento de valores vacíos
# Explicación: rellenamos numéricos con mediana (robusto a outliers) y
# categóricos con la moda.
# ---------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

for col in numeric_cols:
    nnull = df[col].isnull().sum()
    if nnull:
        df[col].fillna(df[col].median(), inplace=True)
        print(f"Rellenado numérico: {col} ({nnull} valores) -> mediana")

for col in cat_cols:
    nnull = df[col].isnull().sum()
    if nnull:
        df[col].fillna(df[col].mode()[0], inplace=True)
        print(f"Rellenado categórico: {col} ({nnull} valores) -> moda")

# Nota: si 'Sleep Disorder' usa NaN para "No disorder" muchos notebooks lo convierten a 'No Sleep Disorder'.
if 'Sleep Disorder' in df.columns:
    df['Sleep Disorder'] = df['Sleep Disorder'].fillna('No Sleep Disorder')

# ---------------------------
# Análisis univariante
# - Distribuciones de numéricas
# - Conteo de categóricas
# Explicación: ver sesgo, asimetría, concentraciones para decidir transformaciones.
# ---------------------------
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Histogramas (solo hasta 12 variables para evitar saturación)
n_plot = min(len(num_cols), 12)
fig, axes = plt.subplots((n_plot+2)//3, 3, figsize=(14, 4*((n_plot+2)//3)))
axes = axes.ravel()
for i, c in enumerate(num_cols[:n_plot]):
    axes[i].hist(df[c].dropna(), bins=30, alpha=0.75, edgecolor='k')
    axes[i].set_title(c)
plt.tight_layout()
plt.savefig('imagenes/distribuciones_numericas.png', dpi=200)
plt.close()

# Categóricas: top categories
fig, axes = plt.subplots((min(len(cat_cols),6)+2)//3, 3, figsize=(14, 4*((min(len(cat_cols),6)+2)//3)))
axes = axes.ravel()
for i, c in enumerate(cat_cols[:6]):
    df[c].value_counts().plot(kind='bar', ax=axes[i])
    axes[i].set_title(c)
    axes[i].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('imagenes/distribuciones_categoricas.png', dpi=200)
plt.close()

# ---- Deducciones ejemplo para Stress_Level si existe
if 'Stress Level' in df.columns or 'Stress_Level' in df.columns:
    sc = 'Stress Level' if 'Stress Level' in df.columns else 'Stress_Level'
    print("\nAnálisis resumen de", sc)
    print(df[sc].describe())
    print("Media: %.2f - Mediana: %.2f - Std: %.2f" % (df[sc].mean(), df[sc].median(), df[sc].std()))

# ---------------------------
# Filtrado de outliers (IQR)
# Explicación: eliminamos valores extremos que pudieran distorsionar modelos.
# Procedimiento: para cada variable numérica (excluimos la variable 'Stress' por ahora),
# construimos una máscara que mantiene filas dentro de [Q1-1.5IQR, Q3+1.5IQR].
# ENFOQUE: calculamos máscara combinada (AND) para evitar eliminar muchas filas sucesivamente.
# ---------------------------
stress_col = None
if 'Stress Level' in df.columns:
    stress_col = 'Stress Level'
elif 'Stress_Level' in df.columns:
    stress_col = 'Stress_Level'

num_cols_for_outlier = [c for c in num_cols if c != stress_col]

# construimos mascara que inicialmente es True
mask = pd.Series(True, index=df.index)
outlier_report = {}
for c in num_cols_for_outlier:
    Q1 = df[c].quantile(0.25)
    Q3 = df[c].quantile(0.75)
    IQR = Q3 - Q1
    lb = Q1 - 1.5*IQR
    ub = Q3 + 1.5*IQR
    mask_col = df[c].between(lb, ub)
    removed = (~mask_col).sum()
    outlier_report[c] = int(removed)
    # aplicamos el filtro combinándolo (se puede cambiar a OR si se desea menos agresivo)
    mask &= mask_col

print("\nOutliers detectados por variable (contados):")
for k, v in outlier_report.items():
    print(f"  - {k}: {v}")

before = len(df)
df = df[mask].copy()
after = len(df)
print(f"\nFilas eliminadas por IQR multivariante: {before - after}  (dataset: {after} filas)")

# Explicación en línea:
# Elegimos IQR multivariante (AND) para asegurar que fila con outlier en varias variables solo se quite una vez.
# Alternativa menos agresiva: aplicar IQR por variable y eliminar solo si excede en N variables.

# ---------------------------
# Creación de variable objetivo binaria
# Según tu especificación:
#   3-6 -> 'ESTRES_MODERADO'
#   7+  -> 'ESTRESADO'
# Decisión práctica (explicada): Si existen valores <3 (muy bajo estrés),
# para mantener una variable estrictamente binaria y consistente con la regla,
# vamos a **filtrar** y conservar solo filas con Stress >= 3.
# Razonamiento: la regla que diste define categorías a partir de 3. Mantener <3 introduciría una categoría
# no contemplada por la regla binaria. Si prefieres otra opción (ej. agrupar <3 con 'ESTRES_MODERADO'),
# puedo ajustar el código.
# ---------------------------
if stress_col is None:
    raise KeyError("No se encontró columna de nivel de estrés. Revisar nombre de columna (Stress Level o Stress_Level).")

# Convertir a numérico por si viene como object
df[stress_col] = pd.to_numeric(df[stress_col], errors='coerce')
initial_len = len(df)
df = df[df[stress_col] >= 3].copy()
filtered_len = len(df)
print(f"\nFilas removidas por tener Stress < 3: {initial_len - filtered_len} (para crear target binaria según regla)")

df['Stress_Category'] = df[stress_col].apply(lambda x: 'ESTRES_MODERADO' if 3 <= x <= 6 else 'ESTRESADO')

print("\nCuenta por categoría:")
print(df['Stress_Category'].value_counts())
print("\nProporciones (%):")
print(df['Stress_Category'].value_counts(normalize=True) * 100)

# Eliminar variable numérica original (especificación del ejercicio)
df.drop(columns=[stress_col], inplace=True)
print("\nVariable numérica de estrés eliminada del dataset (por petición).")

# ---------------------------
# Análisis bivariante: target vs todas las variables
# Guardamos boxplots para numéricas y barras/porcentajes para categóricas
# ---------------------------
num_after = df.select_dtypes(include=[np.number]).columns.tolist()
cat_after = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Stress_Category' in cat_after:
    cat_after = [c for c in cat_after if c != 'Stress_Category']

# Boxplots numéricas por categoría
n_plot = min(len(num_after), 12)
if n_plot > 0:
    fig, axes = plt.subplots((n_plot+2)//3, 3, figsize=(14, 4*((n_plot+2)//3)))
    axes = axes.ravel()
    for i, c in enumerate(num_after[:n_plot]):
        sns.boxplot(x='Stress_Category', y=c, data=df, ax=axes[i])
        axes[i].set_title(f'{c} vs Stress_Category')
    plt.tight_layout()
    plt.savefig('imagenes/bivariante_numericas.png', dpi=200)
    plt.close()

# Para categóricas: proporciones dentro de cada Stress_Category
for c in cat_after[:6]:
    plt.figure(figsize=(6,4))
    ct = pd.crosstab(df[c], df['Stress_Category'], normalize='index') * 100
    ct.plot(kind='bar', stacked=False)
    plt.title(f'{c} por Stress_Category (% por fila)')
    plt.ylabel('% por categoría de "{0}"'.format(c))
    plt.tight_layout()
    plt.savefig(f'imagenes/bivariante_{c}.png', dpi=200)
    plt.close()

# ---------------------------
# Matriz de correlación (codificamos target)
# ---------------------------
df['Stress_Encoded'] = df['Stress_Category'].map({'ESTRES_MODERADO':0, 'ESTRESADO':1})
corr_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[corr_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matriz de correlación (numéricas)')
plt.tight_layout()
plt.savefig('imagenes/matriz_correlacion.png', dpi=200)
plt.close()

# Reporte correlaciones altas
high_corr_pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        val = corr.iloc[i,j]
        if abs(val) > 0.7:
            high_corr_pairs.append((corr.columns[i], corr.columns[j], val))
if high_corr_pairs:
    print("\nCorrelaciones fuertes (|r|>0.7):")
    for a,b,v in high_corr_pairs:
        print(f"  - {a} <-> {b}: {v:.2f}")
else:
    print("\nNo se detectaron correlaciones muy fuertes (|r|>0.7).")

# Si hay variables muy correlacionadas entre sí (no objetivo), recomendar eliminar la que "tenga menos sentido"
# Ejemplo comentado:
# if 'Cost_of_Living' and 'Salary' correlacionan fuertemente, podríamos eliminar 'Cost_of_Living' si consideramos 'Salary' más relevante.

# ---------------------------
# División Train/Test 80/20 estratificada
# ---------------------------
X = df.drop(columns=['Stress_Category', 'Stress_Encoded'])
y = df['Stress_Category']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain: {len(X_train)} | Test: {len(X_test)}")
print("\nProporciones en Train (%):")
print(y_train.value_counts(normalize=True) * 100)
print("\nProporciones en Test (%):")
print(y_test.value_counts(normalize=True) * 100)

train_df = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
test_df = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)
train_df.to_csv('datos/train.csv', index=False)
test_df.to_csv('datos/test.csv', index=False)
print("\nGuardados: datos/train.csv, datos/test.csv")

# Guardado final del dataset procesado (por si se requiere)
df.to_csv('datos/processed_full.csv', index=False)
print("Guardado: datos/processed_full.csv")

print("\n✅ EDA completado. Revise la carpeta 'imagenes/' para gráficas y 'datos/' para CSVs.")
